In [11]:
import time
import pandas as pd
import requests

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


# Headers for API Request
header_data  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

# constants for season type
regular_season = 'Regular Season'
playoffs = 'Playoffs'

# build play by play url
def play_by_play_url(game_id):
    return "https://stats.nba.com/stats/playbyplayv2/?gameId={0}&startPeriod=0&endPeriod=14".format(game_id)

# build game log url
def game_log_url(season, season_type):
    return "http://stats.nba.com/stats/leaguegamelog/?leagueId=00&season={}&seasonType={}&playerOrTeam=T&counter=0&sorter=PTS&direction=ASC&dateFrom=&dateTo=".format(season, season_type)

# extracts pbp data from api response
def extract_data(url):
    print(url)
    r = requests.get(url, headers=header_data)
    resp = r.json()
    results = resp['resultSets'][0]
    headers = results['headers']
    rows = results['rowSet']
    frame = pd.DataFrame(rows)
    frame.columns = headers
    return frame


###
### Download and Save Play by Play Data
###
def download_pbp(game_id):
    play_by_play = extract_data(play_by_play_url(game_id))

    play_by_play.to_csv("data2/{}_pbp.csv".format(game_id), index=False)

# # Set the season and season typ
season = '2021-22'

### SET THE SEASON ABOVE ###

season_type = regular_season

# # Download the league game log
schedule = extract_data(game_log_url(season, season_type))

teams = ['Boston Celtics', 'New York Knicks', 'Toronto Raptors', 'Philadelphia 76ers', 'Brooklyn Nets']

schedule = schedule[schedule['TEAM_NAME'].isin(teams)]
# # Save the game log in case you want it for future reference
# schedule.to_csv('data/schedule_{}_{}.csv'.format(season, season_type), index=False)
# # Get all of the unique game ids
game_ids = schedule['GAME_ID'].unique()
game_ids.sort()


#########################

#Run this to get game ids

#########################

game_ids
# # For each game id, download the play by play then sleep for .75 seconds so that we don't hit the stats nba rate limit
# for id in game_ids[303:]:
#     download_pbp(id)
#     time.sleep(.75)

# Need to run this once for the first time...it will take...a while
# Commented so I don't actually run it

http://stats.nba.com/stats/leaguegamelog/?leagueId=00&season=2021-22&seasonType=Regular Season&playerOrTeam=T&counter=0&sorter=PTS&direction=ASC&dateFrom=&dateTo=
https://stats.nba.com/stats/playbyplayv2/?gameId=0022101022&startPeriod=0&endPeriod=14
https://stats.nba.com/stats/playbyplayv2/?gameId=0022101028&startPeriod=0&endPeriod=14
https://stats.nba.com/stats/playbyplayv2/?gameId=0022101030&startPeriod=0&endPeriod=14
https://stats.nba.com/stats/playbyplayv2/?gameId=0022101034&startPeriod=0&endPeriod=14
https://stats.nba.com/stats/playbyplayv2/?gameId=0022101036&startPeriod=0&endPeriod=14
https://stats.nba.com/stats/playbyplayv2/?gameId=0022101037&startPeriod=0&endPeriod=14
https://stats.nba.com/stats/playbyplayv2/?gameId=0022101042&startPeriod=0&endPeriod=14
https://stats.nba.com/stats/playbyplayv2/?gameId=0022101044&startPeriod=0&endPeriod=14
https://stats.nba.com/stats/playbyplayv2/?gameId=0022101046&startPeriod=0&endPeriod=14
https://stats.nba.com/stats/playbyplayv2/?gameId=00221

In [27]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Game Id
# c=302

#index 302 not in there
# for game_id in game_ids[303:]:
#     print(game_id)
#     print(c)
#     c+=1
#     if game_id in ['0022100037', '0022100081', '0022100159',
#                   '0022100384','0022100385','0022100399']:
#         continue
# for game_id in ["0022100005"]:
# for game_id in ["0022100291"]:

# Uncomment the following to use - if certain game ids don't work, thenyou can indent the stuff inside the loop and use
# if game_id not in [list of broken ids]

# for game_id in game_ids: ### DO NOT RUN THIS UNLESS ITS THE FIRST TIME - can uncomment this and run - will download files
# and you'll need to keep them in the correct directory.

    # Headers for API Request
    header_data = {
        'Host': 'stats.nba.com',
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36',
        'Referer': 'stats.nba.com',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    }


    ###
    ### Calculate Players on court at the start of each period
    ###

    # Build advanced boxscore url
    def advanced_boxscore_url(game_id, start, end):
        return 'https://stats.nba.com/stats/boxscoretraditionalv2/?gameId={0}&startPeriod=0&endPeriod=14&startRange={1}&endRange={2}&rangeType=2'.format(
            game_id, start, end)


    # Helper functions
    def calculate_time_at_period(period):
        if period > 5:
            return (720 * 4 + (period - 5) * (5 * 60)) * 10
        else:
            return (720 * (period - 1)) * 10


    def split_subs(df, tag):
        subs = df[[tag, 'PERIOD', 'EVENTNUM']]
        subs['SUB'] = tag
        subs.columns = ['PLAYER_ID', 'PERIOD', 'EVENTNUM', 'SUB']
        return subs


    def frame_to_row(df):
        team1 = df['TEAM_ID'].unique()[0]
        team2 = df['TEAM_ID'].unique()[1]
        players1 = df[df['TEAM_ID'] == team1]['PLAYER_ID'].tolist()
        players1.sort()
        players2 = df[df['TEAM_ID'] == team2]['PLAYER_ID'].tolist()
        players2.sort()

        lst = [team1]
        lst.append(players1)
        lst.append(team2)
        lst.append(players2)


        return lst


    # extracts data from api response
    def extract_data(url):
        print(url)
        r = requests.get(url, headers=header_data)
        resp = r.json()
        results = resp['resultSets'][0]
        headers = results['headers']
        rows = results['rowSet']
        frame = pd.DataFrame(rows)
        frame.columns = headers
        return frame


    play_by_play = pd.read_csv(('data2/{}_pbp.csv'.format(game_id)))

    substitutionsOnly = play_by_play[(play_by_play['EVENTMSGTYPE'] == 8)][['PERIOD','EVENTNUM','PLAYER1_ID','PLAYER2_ID']]
    substitutionsOnly.columns = ['PERIOD', 'EVENTNUM', 'OUT', 'IN']

    subs_in = split_subs(substitutionsOnly, 'IN')
    subs_out = split_subs(substitutionsOnly, 'OUT')

    full_subs = pd.concat([subs_out, subs_in], axis=0).reset_index()[['PLAYER_ID', 'PERIOD', 'EVENTNUM', 'SUB']]
    first_event_of_period = full_subs.loc[full_subs.groupby(by=['PERIOD', 'PLAYER_ID'])['EVENTNUM'].idxmin()]
    players_subbed_in_at_each_period = first_event_of_period[first_event_of_period['SUB'] == 'IN'][
        ['PLAYER_ID', 'PERIOD', 'SUB']]

    
    periods = players_subbed_in_at_each_period['PERIOD'].drop_duplicates().values.tolist()

    if game_id == "0022100291":
        periods = [1,2,3,4,5,6,7]
    
    rows = []
    for period in periods:
        low = calculate_time_at_period(period) + 5
        high = calculate_time_at_period(period + 1) - 5
        boxscore = advanced_boxscore_url(game_id, low, high)
        try:
            boxscore_players = extract_data(boxscore)[['PLAYER_NAME', 'PLAYER_ID', 'TEAM_ID']]
        except ValueError:
            continue
        boxscore_players['PERIOD'] = period

        players_subbed_in_at_period = players_subbed_in_at_each_period[players_subbed_in_at_each_period['PERIOD'] == period]

        joined_players = pd.merge(boxscore_players, players_subbed_in_at_period, on=['PLAYER_ID', 'PERIOD'], how='left')
        joined_players = joined_players[pd.isnull(joined_players['SUB'])][['PLAYER_NAME', 'PLAYER_ID', 'TEAM_ID', 'PERIOD']]
        row = frame_to_row(joined_players)
        row.append(period)
        rows.append(row)
        time.sleep(.25)
    print(rows)
    players_on_court_at_start_of_period = pd.DataFrame(rows)
    cols = ['TEAM_ID_1', 'TEAM_1_PLAYERS', 'TEAM_ID_2', 'TEAM_2_PLAYERS', 'PERIOD']
    players_on_court_at_start_of_period.columns = cols
    players_on_court_at_start_of_period.to_csv("data2/{}_players_at_period.csv".format(game_id), index=False)
    print(game_id)
    
#pull and extract game data

0022101022
302
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101022&startPeriod=0&endPeriod=14&startRange=5&endRange=7195&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101022&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101022&startPeriod=0&endPeriod=14&startRange=14405&endRange=21595&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101022&startPeriod=0&endPeriod=14&startRange=21605&endRange=28795&rangeType=2
[[1610612743, [201145, 203115, 203932, 203999, 1628420], 1610612755, [201935, 202699, 203954, 1629680, 1630178], 1], [1610612743, [202326, 203085, 203210, 1627854, 1630538], 1610612755, [201980, 202699, 203954, 1627777, 1630178], 2], [1610612743, [201145, 203115, 203932, 203999, 1628420], 1610612755, [201935, 202699, 203954, 1629680, 1630178], 3], [1610612743, [202326, 203085, 203210, 1627854, 1630538], 1610612755, [201980, 203

https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101046&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101046&startPeriod=0&endPeriod=14&startRange=14405&endRange=21595&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101046&startPeriod=0&endPeriod=14&startRange=21605&endRange=28795&rangeType=2
[[1610612742, [203915, 203939, 1627827, 1628973, 1629029], 1610612755, [201935, 202699, 203954, 1629680, 1630178], 1], [1610612742, [202722, 203915, 1628467, 1628973, 1630182], 1610612755, [201599, 201935, 201980, 1627777, 1629003], 2], [1610612742, [203915, 203939, 1627827, 1628973, 1629029], 1610612755, [201935, 202699, 203954, 1629680, 1630178], 3], [1610612742, [203915, 203939, 1627827, 1628973, 1630182], 1610612755, [201599, 201935, 201980, 1627777, 1629003], 4]]
0022101046
0022101048
311
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101048&startPeriod=0

https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101074&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101074&startPeriod=0&endPeriod=14&startRange=14405&endRange=21595&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101074&startPeriod=0&endPeriod=14&startRange=21605&endRange=28795&rangeType=2
[[1610612748, [200768, 200782, 202710, 1628389, 1629130], 1610612755, [200794, 202699, 1627777, 1629680, 1630178], 1], [1610612748, [200768, 202693, 1628997, 1629622, 1629639], 1610612755, [201599, 201980, 1627777, 1627788, 1629003], 2], [1610612748, [200768, 200782, 202710, 1628389, 1629130], 1610612755, [200794, 202699, 1627777, 1629680, 1630178], 3], [1610612748, [200768, 1628389, 1628997, 1629622, 1629639], 1610612755, [201599, 201980, 1627777, 1627788, 1629003], 4]]
0022101074
0022101075
319
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101075&startPerio

https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101093&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101093&startPeriod=0&endPeriod=14&startRange=14405&endRange=21595&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101093&startPeriod=0&endPeriod=14&startRange=21605&endRange=28795&rangeType=2
[[1610612755, [201935, 202699, 203954, 1629680, 1630178], 1610612747, [2730, 201566, 1626169, 1629117, 1630559], 1], [1610612755, [201599, 201980, 1627777, 1629003, 1630178], 1610612747, [2546, 201566, 201571, 1628370, 1629117], 2], [1610612755, [201935, 202699, 203954, 1629680, 1630178], 1610612747, [2730, 201566, 1626169, 1629117, 1630559], 3], [1610612755, [201599, 202699, 1627788, 1629003, 1630178], 1610612747, [2546, 201566, 201571, 1628370, 1629117], 4]]
0022101093
0022101095
327
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101095&startPeriod=0&endPeri

https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101118&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101118&startPeriod=0&endPeriod=14&startRange=14405&endRange=21595&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101118&startPeriod=0&endPeriod=14&startRange=21605&endRange=28795&rangeType=2
[[1610612755, [201935, 202699, 203954, 1629680, 1630178], 1610612756, [101108, 203109, 1626164, 1628969, 1629028], 1], [1610612755, [201599, 201980, 202699, 1629003, 1630178], 1610612756, [101108, 202687, 1626166, 1628470, 1629013], 2], [1610612755, [201935, 202699, 203954, 1629680, 1630178], 1610612756, [101108, 203109, 1626164, 1628969, 1629028], 3], [1610612755, [201599, 201935, 202699, 1627777, 1629003], 1610612756, [101108, 202687, 1628470, 1628969, 1629013], 4]]
0022101118
0022101120
335
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101120&startPeriod=0

https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101141&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101141&startPeriod=0&endPeriod=14&startRange=14405&endRange=21595&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101141&startPeriod=0&endPeriod=14&startRange=21605&endRange=28795&rangeType=2
[[1610612750, [201976, 1626156, 1626157, 1629020, 1630162], 1610612761, [1627783, 1627832, 1628384, 1629018, 1630567], 1], [1610612750, [1626156, 1627752, 1629162, 1629669, 1629675], 1610612761, [201152, 1627783, 1628384, 1630173, 1630567], 2], [1610612750, [201976, 1626156, 1626157, 1629020, 1630162], 1610612761, [1627783, 1627832, 1628384, 1629018, 1630173], 3], [1610612750, [1626156, 1627752, 1629162, 1629669, 1629675], 1610612761, [201152, 1627783, 1628384, 1628449, 1630567], 4]]
0022101141
0022101147
343
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=00221011

https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101166&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101166&startPeriod=0&endPeriod=14&startRange=14405&endRange=21595&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101166&startPeriod=0&endPeriod=14&startRange=21605&endRange=28795&rangeType=2
[[1610612764, [203107, 203484, 204001, 1629060, 1630557], 1610612738, [201143, 203935, 1627759, 1628369, 1628464], 1], [1610612764, [202397, 1629655, 1630166, 1630264, 1630557], 1610612738, [1628369, 1628401, 1628464, 1629684, 1630202], 2], [1610612764, [203107, 203484, 204001, 1629060, 1630557], 1610612738, [201143, 203935, 1627759, 1628369, 1628464], 3], [1610612764, [202397, 1629655, 1630166, 1630264, 1630557], 1610612738, [201143, 1627759, 1628401, 1629684, 1630202], 4]]
0022101166
0022101170
351
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101170&startP

https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101193&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101193&startPeriod=0&endPeriod=14&startRange=14405&endRange=21595&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101193&startPeriod=0&endPeriod=14&startRange=21605&endRange=28795&rangeType=2
[[1610612738, [201143, 203935, 1627759, 1628369, 1628464], 1610612741, [201942, 202696, 203897, 1630172, 1630245], 1], [1610612738, [1628369, 1628401, 1628464, 1629684, 1630202], 1610612741, [201942, 202684, 1627884, 1629632, 1629750], 2], [1610612738, [201143, 203935, 1627759, 1628369, 1628464], 1610612741, [201942, 202696, 203897, 1630172, 1630245], 3], [1610612738, [1627759, 1628401, 1628464, 1629684, 1630202], 1610612741, [201942, 202684, 1627884, 1629632, 1629750], 4]]
0022101193
0022101197
359
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101197&startPe

https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101223&startPeriod=0&endPeriod=14&startRange=7205&endRange=14395&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101223&startPeriod=0&endPeriod=14&startRange=14405&endRange=21595&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101223&startPeriod=0&endPeriod=14&startRange=21605&endRange=28795&rangeType=2
[[1610612738, [201143, 1627759, 1629684, 1630174, 1630202], 1610612763, [203937, 1629001, 1629633, 1630582, 1630583], 2], [1610612738, [201143, 203935, 1627759, 1628369, 1628464], 1610612763, [203937, 1629001, 1629723, 1630214, 1630533], 3], [1610612738, [1628436, 1629684, 1630174, 1630202, 1630573], 1610612763, [203937, 1629001, 1629633, 1630582, 1630583], 4]]
0022101223
0022101226
367
https://stats.nba.com/stats/boxscoretraditionalv2/?gameId=0022101226&startPeriod=0&endPeriod=14&startRange=5&endRange=7195&rangeType=2
https://stats.nba.com/stats/boxscoretraditionalv2/?gameI

In [89]:
not_working = ['0022100005', "0022100019", "0022100107", "0022100158", 
                       "0022100182", "0022100225", "0022100239",
                      "0022100251", "0022100275", "0022100493",'0022100078',
                      '0022100287', '0022100510', '0022100568', '0022100737', '0022100748',
                      '0022100850', '0022100851','0022100901', '0022100905', '0022100856', '0022100906',
                      '0022100947','0022100981','0022100989','0022101014','0022101015',
                      '0022101216', '0022101223']
game_ids = [game for game in game_ids if game not in not_working]
    
    

In [87]:
# Import os for relative pathing to data
import os
import pandas as pd
# Import our play by play utils file
from play_by_play_utils import *

# Set columns and width for easier printing
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

parsed_possessions = []

# UnComment when needed! 

# for game_id in game_ids:
    dirname = ""

    # generate file path for play by play and players on court data
    input_play_by_play = os.path.join(dirname, './data2/{}_pbp.csv'.format(game_id))
    input_players_on_court = os.path.join(dirname, './data2/{}_players_at_period.csv'.format(game_id))
    output_path = os.path.join(dirname, './data2/{}_possessions.csv'.format(game_id))


    # Read in play by play and fill null description columsn with empty string
    play_by_play = pd.read_csv(input_play_by_play, index_col=False)
    play_by_play[home_description] = play_by_play[home_description].fillna("")
    play_by_play[neutral_description] = play_by_play[home_description].fillna("")
    play_by_play[away_description] = play_by_play[away_description].fillna("")


    # We will need to know the game clock at each event later on. Let's take the game clock string (7:34) and convert it into seconds elapsed
    def parse_time_elapsed(time_str, period):
        # Maximum minutes in a period is 12 unless overtime
        max_minutes = 12 if period < 5 else 5
        # Split string on :
        [minutes, sec] = time_str.split(':')
        # extract minutes and seconds
        minutes = int(minutes)
        sec = int(sec)

        # 7:34 (4 minutes 26 seconds have passed) -> 12 - 7 -> 5, need to subtract an extra minute.
        min_elapsed = max_minutes - minutes - 1
        sec_elapsed = 60 - sec

        return (min_elapsed * 60) + sec_elapsed

    # We will also need to calculate the total time elapsed, not just the time elapsed in the period
    def calculate_time_elapsed(row):
        # Caclulate time elapsed in the period
        time_in_period = calculate_time_elapsed_period(row)
        period = row[period_column]
        # Calculate total time elapsed up to the start of the current period
        if period > 4:
            return (12 * 60 * 4) + ((period - 5) * 5 * 60) + time_in_period
        else:
            return ((period - 1) * 12 * 60) + time_in_period

    # method for calculating time elapsed in a period from a play by play event row
    def calculate_time_elapsed_period(row):
        return parse_time_elapsed(row[game_clock], row[period_column])

    # Apply the methods for calculating time to add the columns to the dataframe
    play_by_play[time_elapsed] = play_by_play.apply(calculate_time_elapsed, axis=1)
    play_by_play[time_elapsed_period] = play_by_play.apply(calculate_time_elapsed_period, axis=1)


    # Read the players at the start of each period
    try:
        players_at_start_of_period = pd.read_csv(input_players_on_court)
    except FileNotFoundError:
        continue
    # Players at the start of each period are stored as an string in the dataframe column
    # We need to parse out that string into an array of player Ids
    def split_row(list_str):
        return [x.replace('[', '').replace(']', '').strip() for x in list_str.split(',')]

    # We need to keep track of substitutions as they happen. To do this we will maintain a map of players on the court at a given moment
    # It will be structured as period -> team_id -> players array
    sub_map = {}
    # Pre-populate the map with the players at the start of each period
    for row in players_at_start_of_period.iterrows():
        sub_map[row[1][period_column]] = {row[1]['TEAM_ID_1']: split_row(row[1]['TEAM_1_PLAYERS']),
                                     row[1]['TEAM_ID_2']: split_row(row[1]['TEAM_2_PLAYERS'])}

    # players on the court need to be updated after every substitution
    def update_subs(row):
        period = row[period_column]
        # If the event is a substitution we need to sub out the players on the court
        if is_substitution(row):
            team_id = row[player1_team_id]
            player_in = str(row[player2_id])
            player_out = str(row[player1_id])
            players = sub_map[period][team_id]
            players_index = players.index(player_out)
            players[players_index] = player_in
            players.sort()
            sub_map[period][team_id] = players
        # Once we have subbed the players in/out in our sub map we can add those players to the current event row so that
        # Each event has all of the players involved with the event included
        for i, k in enumerate(sub_map[period].keys()):
            row['TEAM{}_ID'.format(i + 1)] = k
            row['TEAM{}_PLAYER1'.format(i + 1)] = sub_map[period][k][0]
            row['TEAM{}_PLAYER2'.format(i + 1)] = sub_map[period][k][1]
            row['TEAM{}_PLAYER3'.format(i + 1)] = sub_map[period][k][2]
            row['TEAM{}_PLAYER4'.format(i + 1)] = sub_map[period][k][3]
            row['TEAM{}_PLAYER5'.format(i + 1)] = sub_map[period][k][4]
    print(game_id)

    """
    What ends a possession?


    1. Made Shot (Need to account for And-1s)
    2. Defensive Rebound
    3. Turnover
    4. Last made free throw  (Ignore FT 1 of 1 on away from play fouls with no made shot)
    """
    def is_end_of_possession(ind, row, rows):
        return is_turnover(row) or (is_last_free_throw_made(ind, row, rows)) or is_defensive_rebound(ind, row, rows) or \
               is_make_and_not_and_1(ind, row, rows) or is_end_of_period(row)


    # The main function of our tutorial, the method to group events by possession
    def parse_possessions(rows):
        # we will have a list of possessions and each possession will be a list of events
        possessions = []
        current_posession = []
        for ind, row in rows:
            # update our subs
            update_subs(row)
            # No need to include subs or end of period events in our possession list
            if not is_substitution(row) and not is_end_of_period(row):
                current_posession.append(row)
            # if the current event is the last event of a possession, add the current possession to our list of possessions
            # and start a new possession
            if is_end_of_possession(ind, row, rows):
                # No need to add empty end of period possessions
                if len(current_posession) > 0:
                    possessions.append(current_posession)
                current_posession = []
        return possessions


    # convert dataframe into a list of rows. I know there is a better way to do this,
    # but this is the first thing I thought of.
    pbp_rows = list(play_by_play.iterrows())
    possessions = parse_possessions(pbp_rows)


    # Print out the first couple of possessions so that you can see how the parser split them.


    # We need to count up each teams points from a possession
    def count_points(possession):
        # points will be a map where the key is the team id and the value is the number of points scored in that possesion
        points = {}
        for p in possession:
            if is_made_shot(p) or (not is_miss(p) and is_free_throw(p)):
                if p[player1_team_id] in points:
                    points[p[player1_team_id]] += extract_points(p)
                else:
                    points[p[player1_team_id]] = extract_points(p)
        return points

    # We need to count up each teams turnovers
    def count_turnovers(possession):
        # turnovers will be a map where the key is the team id and the value is the number of 
        # turnovers scored in that possesion
        # This is how all the following functions will look
        turnovers = {}
        for p in possession:
            if is_turnover(p):
                if p[player1_team_id] in turnovers:
                    turnovers[p[player1_team_id]] += 1
                else:
                    turnovers[p[player1_team_id]] = 1
        return turnovers

    # COUNT REBOUNDS
    def count_rebounds(possession):
        rebounds = {}
        for p in possession:
            if is_rebound(p):
                if p[player1_team_id] in rebounds:
                    rebounds[p[player1_team_id]] += 1
                else:
                    rebounds[p[player1_team_id]] = 1
        return rebounds

    # COUNT BLOCKS
    def count_blocks(possession):
        blocks = {}
        for p in possession:
            if is_block(p):
                if p[player1_team_id] in blocks:
                    blocks[p[player1_team_id]] += 1
                else:
                    blocks[p[player1_team_id]] = 1
        return blocks

    # COUNT Steals
    def count_steals(possession):
        steals = {}
        for p in possession:
            if is_steal(p):
                if p[player1_team_id] in steals:
                    steals[p[player1_team_id]] += 1
                else:
                    steals[p[player1_team_id]] = 1
        return steals

    #Count Field Goals Attempted
    def count_fga(possession):
        fga = {}
        for p in possession:
            if ((is_made_shot(p) or is_missed_shot(p)) and (not is_free_throw(p))):
                if p[player1_team_id] in fga:
                    fga[p[player1_team_id]] += 1
                else:
                    fga[p[player1_team_id]] = 1
        return fga

    #Count 3ptrs attempted
    def count_threeptrs(possession):
        threeptrs = {}
        for p in possession:
            if is_three(p):
                if p[player1_team_id] in threeptrs:
                    threeptrs[p[player1_team_id]] += 1
                else:
                    threeptrs[p[player1_team_id]] = 1
        return threeptrs

    #three pointers made
    def count_threeptrsmade(possession):
        threeptrsmade = {}
        for p in possession:
            if is_three(p) and is_made_shot(p):
                if p[player1_team_id] in threeptrsmade:
                    threeptrsmade[p[player1_team_id]] += 1
                else:
                    threeptrsmade[p[player1_team_id]] = 1
        return threeptrsmade

    # count fg made
    def count_fgmade(possession):
        fgmade = {}
        for p in possession:
            if is_made_shot(p) and not is_free_throw(p):
                if p[player1_team_id] in fgmade:
                    fgmade[p[player1_team_id]] += 1
                else:
                    fgmade[p[player1_team_id]] = 1
        return fgmade

    # free throws attempted
    def count_fta(possession):
        fta = {}
        for p in possession:
            if ((is_made_shot(p) or is_missed_shot(p)) and (is_free_throw(p))):
                if p[player1_team_id] in fta:
                    fta[p[player1_team_id]] += 1
                else:
                    fta[p[player1_team_id]] = 1
        return fta

    # free throws made
    def count_ftmade(possession):
        ftmade = {}
        for p in possession:
            if is_made_shot(p) and is_free_throw(p):
                if p[player1_team_id] in ftmade:
                    ftmade[p[player1_team_id]] += 1
                else:
                    ftmade[p[player1_team_id]] = 1
        return ftmade


    # We need to know how many points each shot is worth:
    def extract_points(p):
        if is_free_throw(p) and not is_miss(p):
            return 1
        elif is_made_shot(p) and is_three(p):
            return 3
        elif is_made_shot(p) and not is_three(p):
            return 2
        else:
            return 0

    # We need to determine which team has possession of the ball based on how the possession ended

    # If the possession ended with a made shot or free throw then we can determine that the team of the player
    # who made the shot was the team with possession of the ball
    #
    # If the possession ended with a rebound then we can determine that the team that did not get the rebound is
    # the team that had possession of the ball (ORBDs do not end possessions)
    #
    # If the possession ended with a turnover then we can determine that the team that committed the turnover is
    # the team that had possession of the ball
    #
    # If the possession ended due to the end of a period, we probably have some other random event as the last event
    # We can assume that the team1 id of that event is the team with the ball

    # improvements can be made by handling each event type individually
    def determine_possession_team(p, team1, team2):
        if is_made_shot(p) or is_free_throw(p):
            return str(int(p[player1_team_id]))
        elif is_rebound(p):
            if is_team_rebound(p):
                if p[player1_id] == team1:
                    return team2
                else:
                    return team1
            else:
                if p[player1_team_id] == team1:
                    return team2
                else:
                    return team1
        elif is_turnover(p):
            if is_team_turnover(p):
                return str(int(p[player1_id]))
            else:
                return str(int(p[player1_team_id]))
        else:
            if math.isnan(p[player1_team_id]):
                return str(int(p[player1_id]))
            else:
                return str(int(p[player1_team_id]))


    # Parse out the list of events in a possession into a single possession object
    # for this tutorial we will only include the players on the court, the game id, period, start and end time of possesion,
    # points scored by each team, and which team was on offense during the possession.
    def parse_possession(possession):
        times_of_events = [p[time_elapsed] for p in possession]
        possession_start = min(times_of_events)
        possession_end = max(times_of_events)
        points = count_points(possession)
        turnovers = count_turnovers(possession)
        rebounds = count_rebounds(possession)
        blocks = count_blocks(possession)
        steals = count_steals(possession)
        fg_attempted = count_fga(possession)
        three_pointers_shot = count_threeptrs(possession) 
        three_pointers_made = count_threeptrsmade(possession)
        fgmade = count_fgmade(possession)
        ft_att = count_fta(possession)
        ftmade = count_ftmade(possession)
        game_id = possession[0]['GAME_ID']
        period = possession[0][period_column]
        team1_id = possession[0]['TEAM1_ID']
        team1_player1 = possession[0]['TEAM1_PLAYER1']
        team1_player2 = possession[0]['TEAM1_PLAYER2']
        team1_player3 = possession[0]['TEAM1_PLAYER3']
        team1_player4 = possession[0]['TEAM1_PLAYER4']
        team1_player5 = possession[0]['TEAM1_PLAYER5']
        team1_points = points[team1_id] if team1_id in points else 0
        team1_turnovers = turnovers[team1_id] if team1_id in turnovers else 0
        team1_rebounds = rebounds[team1_id] if team1_id in rebounds else 0
        team1_blocks = blocks[team1_id] if team1_id in blocks else 0
        team1_steals = steals[team1_id] if team1_id in steals else 0
        team1_3ptatt = three_pointers_shot[team1_id] if team1_id in three_pointers_shot else 0
        team1_3ptmade = three_pointers_made[team1_id] if team1_id in three_pointers_made else 0
        team1_fga = fg_attempted[team1_id] if team1_id in fg_attempted else 0
        team1_fgm = fgmade[team1_id] if team1_id in fgmade else 0
        team1_fta = ft_att[team1_id] if team1_id in ft_att else 0
        team1_ftmade = ftmade[team1_id] if team1_id in ftmade else 0
        team2_id = possession[0]['TEAM2_ID']
        team2_player1 = possession[0]['TEAM2_PLAYER1']
        team2_player2 = possession[0]['TEAM2_PLAYER2']
        team2_player3 = possession[0]['TEAM2_PLAYER3']
        team2_player4 = possession[0]['TEAM2_PLAYER4']
        team2_player5 = possession[0]['TEAM2_PLAYER5']
        team2_points = points[team2_id] if team2_id in points else 0
        team2_turnovers = turnovers[team2_id] if team2_id in turnovers else 0
        team2_rebounds = rebounds[team2_id] if team2_id in rebounds else 0
        team2_blocks = blocks[team2_id] if team2_id in blocks else 0
        team2_steals = steals[team2_id] if team2_id in steals else 0
        team2_3ptatt = three_pointers_shot[team2_id] if team2_id in three_pointers_shot else 0
        team2_3ptmade = three_pointers_made[team2_id] if team2_id in three_pointers_made else 0
        team2_fga = fg_attempted[team2_id] if team2_id in fg_attempted else 0
        team2_fgm = fgmade[team2_id] if team2_id in fgmade else 0
        team2_fta = ft_att[team1_id] if team2_id in ft_att else 0
        team2_ftmade = ftmade[team1_id] if team2_id in ftmade else 0
        possession_team = determine_possession_team(possession[-1], team1_id, team2_id)

        return {
            'team1_id': str(team1_id),
            'team1_player1': str(team1_player1),
            'team1_player2': str(team1_player2),
            'team1_player3': str(team1_player3),
            'team1_player4': str(team1_player4),
            'team1_player5': str(team1_player5),
            'team2_id': str(team2_id),
            'team2_player1': str(team2_player1),
            'team2_player2': str(team2_player2),
            'team2_player3': str(team2_player3),
            'team2_player4': str(team2_player4),
            'team2_player5': str(team2_player5),
            'game_id': str(game_id),
            'period': period,
            'possession_start': possession_start,
            'possession_end': possession_end,
            'team1_points': team1_points,
            'team2_points': team2_points,
            'team1_turnovers': team1_turnovers,
            'team2_turnovers': team2_turnovers,
            'team1_rebounds': team1_rebounds,
            'team2_rebounds': team2_rebounds,
            'team1_blocks': team1_blocks,
            'team2_blocks': team2_blocks,
            'team1_steals': team1_steals,
            'team2_steals': team2_steals,
            'team1_fga': team1_fga,
            'team2_fga': team2_fga,
            'team1_fgm': team1_fgm,
            'team2_fgm': team2_fgm,
            'team1_3ptrsatt': team1_3ptatt,
            'team2_3ptrsatt': team2_3ptatt,
            'team1_3ptrsmade': team1_3ptmade,
            'team2_3ptrsmade': team2_3ptmade,
            'team1_fta': team1_fta,
            'team2_fta': team2_fta,
            'team1_ftmade': team1_ftmade,
            'team2_ftmade': team2_ftmade,
            'possession_team': str(possession_team)
        }

    # Build a list of parsed possession objects
#         parsed_possessions = []
    for possession in possessions:
        parsed_possessions.append(parse_possession(possession))

        # Build a dataframe from the list of parsed possession
df = pd.DataFrame(parsed_possessions)

# take out if this doesn't work
output_path = os.path.join(dirname, './data2/all_possessions.csv'.format(game_id))

df.to_csv(output_path, index=False)

# Actually parsing it - outputs to one csv! - need to be able to aggregate other staistics!

0022101226
0022101228
